In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/datascience/kaggle.json ~/.kaggle/kaggle.json

In [5]:
#change premissions
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
#download data
! kaggle datasets download -d aliaamiri/titanic-passengers-and-crew-complete-list

  0% 0.00/36.1k [00:00<?, ?B/s]
100% 36.1k/36.1k [00:00<00:00, 32.2MB/s]


In [7]:
#unzip dataset
! unzip titanic-passengers-and-crew-complete-list.zip

Archive:  titanic-passengers-and-crew-complete-list.zip
  inflating: titanic.csv             


In [8]:
#import things plan on using
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
#load dataset
titanic = pd.read_csv('titanic.csv')

In [10]:
# get some intial info on the dataset
print(titanic.head())
print(titanic.info())
print(titanic.describe())
print(titanic.isnull().sum())



                             name  gender   age class embarked        country  \
0             Abbing, Mr. Anthony    male  42.0   3rd        S  United States   
1       Abbott, Mr. Eugene Joseph    male  13.0   3rd        S  United States   
2     Abbott, Mr. Rossmore Edward    male  16.0   3rd        S  United States   
3  Abbott, Mrs. Rhoda Mary 'Rosa'  female  39.0   3rd        S        England   
4     Abelseth, Miss. Karen Marie  female  16.0   3rd        S         Norway   

   ticketno   fare  sibsp  parch survived  
0    5547.0   7.11    0.0    0.0       no  
1    2673.0  20.05    0.0    2.0       no  
2    2673.0  20.05    1.0    1.0       no  
3    2673.0  20.05    1.0    1.0      yes  
4  348125.0   7.13    0.0    0.0      yes  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2207 entries, 0 to 2206
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      2207 non-null   object 
 1   gender    2207 non-n

In [11]:
# drop sib parc and ticket num columns
titanic = titanic.drop(columns=['sibsp', 'parch','ticketno'])
print(titanic.head()) #now only contains 8 columns



                             name  gender   age class embarked        country  \
0             Abbing, Mr. Anthony    male  42.0   3rd        S  United States   
1       Abbott, Mr. Eugene Joseph    male  13.0   3rd        S  United States   
2     Abbott, Mr. Rossmore Edward    male  16.0   3rd        S  United States   
3  Abbott, Mrs. Rhoda Mary 'Rosa'  female  39.0   3rd        S        England   
4     Abelseth, Miss. Karen Marie  female  16.0   3rd        S         Norway   

    fare survived  
0   7.11       no  
1  20.05       no  
2  20.05       no  
3  20.05      yes  
4   7.13      yes  


In [12]:
print(titanic['country'].nunique())

48


In [13]:
# drop null values in age rows
titanic = titanic.dropna(subset=['age'])
print(titanic.isnull().sum())

#replace null in country with unknown
titanic['country'].fillna('unknown',inplace = True)
print(titanic.isnull().sum())


name          0
gender        0
age           0
class         0
embarked      0
country      81
fare        916
survived      0
dtype: int64
name          0
gender        0
age           0
class         0
embarked      0
country       0
fare        916
survived      0
dtype: int64


In [14]:
#group by embarked and class and fill the null values in fare with the mean for that class and location
grouped_df = titanic.groupby(['embarked', 'class'])
titanic['fare'] = grouped_df['fare'].transform(lambda x: x.fillna(x.mean()))

In [15]:
#replace na for crew with crew bec they obviously didn't pay and see how many still missing in fare
crewlist = ['engineering crew', 'victualling crew',
 'restaurant staff', 'deck crew']
titanic.loc[titanic['class'].isin(crewlist),'fare']='crew'
#replace fare nulls with mean of that embarked and class group
print(titanic['embarked'].unique())
print(titanic['class'].unique())
print(titanic.isnull().sum())


['S' 'C' 'B' 'Q']
['3rd' '2nd' '1st' 'engineering crew' 'victualling crew'
 'restaurant staff' 'deck crew']
name        0
gender      0
age         0
class       0
embarked    0
country     0
fare        9
survived    0
dtype: int64


In [16]:
#turns out most with na values for fare were crew members look at rest to see what are
nan_df = titanic[titanic['fare'].isna()]
print(nan_df)
#all 26 are adults so won't be dealing with potentially cheaper fares for kids and only embarked at b and s with 1 c
#not sure what b is most with b are crew but not all. appaently b is crew the ones with a class were from the shipbuilding company
titanic.loc[titanic['embarked']==('B'),'fare']='crew'
print(titanic.isnull().sum())
#now only 17 nan for fare



                                       name gender   age class embarked  \
45                      Andrews, Mr. Thomas   male  39.0   1st        B   
189             Campbell, Mr. William Henry   male  21.0   2nd        B   
230   Chisholm, Mr. Roderick Robert Crispin   male  43.0   1st        B   
288          Cunningham, Mr. Alfred Fleming   male  22.0   2nd        B   
423                 Frost, Mr. Anthony Wood   male  39.0   2nd        B   
676                      Knight, Mr. Robert   male  41.0   2nd        B   
940                     Parkes, Mr. Francis   male  21.0   2nd        B   
942           Parr, Mr. William Henry Marsh   male  30.0   1st        B   
1256             Watson, Mr. Ennis Hastings   male  19.0   2nd        B   

               country fare survived  
45    Northern Ireland  NaN       no  
189   Northern Ireland  NaN       no  
230           Scotland  NaN       no  
288   Northern Ireland  NaN       no  
423            England  NaN       no  
676   Northern 

In [17]:
#loop over all and if not crew convert fare to integer and round
for index, row in titanic.iterrows():
    # Check if the fare value for the current row is not 'crew'
    if row['fare'] != 'crew':
        # Check if the fare value is numeric before conversion
        #if isinstance(row['fare'], (int, float)):
            # Convert fare value to integer after rounding
            titanic.at[index, 'fare'] = int(round(row['fare']))

In [18]:
#no more null values now export to csv
titanic.to_csv('cleaned_titanic.csv')